<a href="https://colab.research.google.com/github/wilberquito/AMLProject/blob/main/Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Project: Image Classification
## Advanced Machine Learning


> Wilber E. Bermeo Quito 
>
> Judit Quintana Massana
>
> April 2023

In [1]:
import zipfile
from pathlib import Path
import torch
import matplotlib.pyplot as plt

In [2]:
try:
    import google.colab
    IN_COLAB = True
    ! pip install torchvision
    ! pip install torchinfo
    
    from google.colab import drive
    drive.mount('/content/drive')

    !rm -rf test
    data_path = Path('/content/drive/MyDrive/AML/test_unlabelled.zip')
    with zipfile.ZipFile(data_path,"r") as zip_ref:
        zip_ref.extractall("test")

    !rm -rf modular
    data_path = Path('/content/drive/MyDrive/AML/modular.zip')
    with zipfile.ZipFile(data_path,"r") as zip_ref:
        zip_ref.extractall(".")

    # Getting the trained models
    !rm -rf trained
    !cp -rf /content/drive/MyDrive/AML/trained ./trained

    # Getting the predictions
    !rm -rf predictions
    !cp -rf /content/drive/MyDrive/AML/predictions ./predictions
except:
    IN_COLAB = False

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from modular.evaluate import val_step
import modular.models as models
from modular.datasets import TestDataset
import torch
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from modular.utils import set_seeds
import pandas as pd

set_seeds(seed=42)

data_path = "./test/test_unlabelled"
out_dim = 10
n_test = 4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
models = {
    Path('./trained/AMLMAXVIT_T.pth'): models.AMLMAXVIT_T(out_dim),
    Path('./trained/amlresnet101.pth'): models.AMLResnet101(out_dim),
    Path('./trained/efficientnet_v2_s.pth'): models.AMLEfficientNet_V2_S(out_dim),
    Path('./trained/AMLResnet_50W.pth'): models.AMLResnet_50W(out_dim) # THIS IS A ResNext50.
}

Path('./predictions').mkdir(exist_ok=True, parents=True)


for p, m in models.items():

  # Check if prediction is already done
  predict_name = Path('./predictions') / Path(p.name.replace('.pth', '.csv'))

  if predict_name.exists():
    print(f'Skipping prediction - {p.name}')
    continue

  # Make prediction
  data = torch.load(p, map_location=device)
  m.load_state_dict(data['model'])
  m = m.to(device)
  transform = m.transforms
  dataset = TestDataset(data_path, transform)
  dataloader = DataLoader(dataset, batch_size=256, shuffle=False)
  _, probs, labels = val_step(m, dataloader, 
                                    device, out_dim, n_test)
  
  # Serialize prediction to csv
  save_as = p.name.replace('.pth', '.csv')
  df = pd.DataFrame(probs)
  df.index = dataset.file_names
  df.to_csv(save_as)

  # Download prediction 
  if IN_COLAB:
    from google.colab import files
    files.download(save_as)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaxVit_T_Weights.IMAGENET1K_V1`. You can also use `weights=MaxVit_T_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.9/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.I

Skipping prediction - AMLMAXVIT_T.pth
Skipping prediction - amlresnet101.pth
Skipping prediction - efficientnet_v2_s.pth
Skipping prediction - AMLResnet_50W.pth


In [4]:
from modular.ensemble import ensemble
ensemble_csv_name = 'ensemble.csv'
parent_dir = './predictions'

ensemble(parent_dir, ensemble_csv_name, with_probs=True)